# Disk Loading Pipeline (`.npy`, `.npz`, `.parquet`)

Demonstrates `NNdex.from_file(...)` where matrix loading and decoding happens in Rust. `.npz` and `.parquet` require a `key` for the embedding matrix/column.

In [ ]:
from __future__ import annotations

from pathlib import Path
import tempfile

import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

from nndex import NNdex


In [ ]:
rng = np.random.default_rng(101)
rows, dims = 128, 16
matrix = rng.normal(size=(rows, dims)).astype(np.float32)
query = matrix[0]


In [ ]:
tmp = Path(tempfile.mkdtemp(prefix="nndex_disk_demo_"))
npy_path = tmp / "embeddings.npy"
npz_path = tmp / "embeddings.npz"
parquet_path = tmp / "embeddings.parquet"

np.save(npy_path, matrix)
np.savez(npz_path, matrix_store=matrix)

array_col = pa.array(matrix.tolist(), type=pa.list_(pa.float32(), list_size=dims))
pq.write_table(pa.table({"embedding": array_col}), parquet_path)

print(tmp)


In [ ]:
idx_npy = NNdex.from_file(str(npy_path), normalized=False, backend="cpu")
idx_npz = NNdex.from_file(str(npz_path), key="matrix_store", normalized=False, backend="cpu")
idx_parquet = NNdex.from_file(str(parquet_path), key="embedding", normalized=False, backend="cpu")

for name, model in [("npy", idx_npy), ("npz", idx_npz), ("parquet", idx_parquet)]:
    indices, scores = model.search(query, k=3)
    print(name, indices.tolist(), scores.tolist())
